In [1]:
# pip install -r requirements.txt
# python -m ipykernel install --user --name=python3

import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dayofmonth, month, quarter, year, dayofweek, date_format, sum as spark_sum

jar_dir = "/home/bnguyen/Desktop/DE_project/notebooks/jars"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    f"--jars {jar_dir}/hadoop-azure-3.3.6.jar,"
    f"{jar_dir}/azure-storage-8.6.6.jar,"
    f"{jar_dir}/jetty-client-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-http-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-io-9.4.43.v20210629.jar,"
    f"{jar_dir}/mysql-connector-j-9.3.0.jar,"
    f"{jar_dir}/jetty-util-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-util-ajax-9.4.43.v20210629.jar "
    "pyspark-shell"
)


In [2]:
# Init spark session
spark = SparkSession.builder \
    .appName("DW data load") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/09 11:17:53 WARN Utils: Your hostname, lenovo-slim, resolves to a loopback address: 127.0.1.1; using 192.168.199.13 instead (on interface wlp2s0)
25/07/09 11:17:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/09 11:18:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
mysql_url = "jdbc:mysql://localhost:3306/store_dw"
mysql_props = {
    "user": "bnguyen",
    "password": ".Tldccmcbtldck2",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [4]:
# Silver access key
spark.conf.set(
    "fs.azure.account.key.mysilver.blob.core.windows.net",
    "bAthp0pVBfqEtyCvJElSX7MeI7ejSLa6cjuPoMz0Gg/69uzEW01y4URMDXsdFCrkpc9M54cDHnXs+AStj1gExQ=="
)

# Gold
spark.conf.set(
    "fs.azure.account.key.mygold.dfs.core.windows.net",
    "wRPXTwWCVxWwUpavEh62A5wzLdUvRTGeB3tZKP3eRbig7ca8ZN51l0kWS32kcbH/ddQ/jNXBzqDC+AStOzXlyw=="
)

In [ ]:
# 1. DimProduct
products = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Products")
dim_product = products.select(
    col("ProductID"),
    col("Name").alias("ProductName"),
    col("CategoryID"),
    col("SellerID")
)
dim_product.write.jdbc(mysql_url, "DimProduct", mode="overwrite", properties=mysql_props)
# dim_product.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimProduct")

25/07/09 11:18:11 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


In [ ]:
# 2. DimCategory
categories = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/ProductCategories")
dim_category = categories.select(
    col("CategoryID"),
    col("CategoryName")
)
dim_category.write.jdbc(mysql_url, "DimCategory", mode="overwrite", properties=mysql_props)
# dim_category.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimCategory")

In [ ]:
# 3. DimSeller
sellers = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Sellers")
dim_seller = sellers.select(
    col("SellerID"),
    col("Name").alias("SellerName")
)
dim_seller.write.jdbc(mysql_url, "DimSeller", mode="overwrite", properties=mysql_props)
# dim_seller.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimSeller")

In [ ]:
# 4. DimCustomer
customers = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Customers")
dim_customer = customers.select(
    col("CustomerID"),
    col("Name").alias("CustomerName")
)
dim_customer.write.jdbc(mysql_url, "DimCustomer", mode="overwrite", properties=mysql_props)
# dim_customer.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimCustomer")

In [ ]:
# 5. DimOrderStatus
order_status = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/OrderStatus")
dim_order_status = order_status.select(
    col("StatusID"),
    col("StatusName")
)
dim_order_status.write.jdbc(mysql_url, "DimOrderStatus", mode="overwrite", properties=mysql_props)
# dim_order_status.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimOrderStatus")

In [ ]:
# 6. DimDate (from Orders)
orders = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Orders")
dim_date = orders.select(
    date_format(col("CreatedAt"), "yyyyMMdd").cast("int").alias("DateKey"),
    col("CreatedAt").cast("date").alias("Date"),
    dayofmonth(col("CreatedAt")).alias("Day"),
    month(col("CreatedAt")).alias("Month"),
    quarter(col("CreatedAt")).alias("Quarter"),
    year(col("CreatedAt")).alias("Year"),
    dayofweek(col("CreatedAt")).alias("DayOfWeek")
).distinct()
dim_date.write.jdbc(mysql_url, "DimDate", mode="overwrite", properties=mysql_props)
# dim_date.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimDate")

In [ ]:
# 7. DimReason
reasons = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Reasons")
dim_reason = reasons.select(
    col("ReasonID"),
    col("ReasonType"),
    col("ReasonDescription")
)
dim_reason.write.jdbc(mysql_url, "DimReason", mode="overwrite", properties=mysql_props)
# dim_reason.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/DimReason")

In [ ]:
# Fact sales
order_items = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/OrderItems").alias("oi")
orders = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Orders").alias("o")
payments = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Payments").alias("p")
products = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Products").alias("pr")
inventory = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Inventory").alias("inv")  # Add this line

# Only consider orders that have a payment
paid_orders = payments.select("OrderID").distinct().alias("po")

# Join OrderItems with Orders and filter for paid orders
fact_sales = (
    order_items
    .join(orders, col("oi.OrderID") == col("o.OrderID"))
    .join(paid_orders, col("oi.OrderID") == col("po.OrderID"), "inner")
    .join(products, col("oi.ProductID") == col("pr.ProductID"))
    .join(inventory, col("oi.ProductID") == col("inv.ProductID"), "left")  # Join with Inventory for UnitCost
    .join(payments, col("oi.OrderID") == col("p.OrderID"), "inner")  # Join with payments
    .select(
        col("oi.OrderItemID"),
        col("oi.OrderID"),
        col("oi.ProductID"),
        col("pr.SellerID"),
        col("o.CustomerID"),
        col("pr.CategoryID"),
        date_format(col("o.CreatedAt"), "yyyyMMdd").cast("int").alias("OrderDateKey"),
        col("o.StatusID"),
        col("oi.Quantity").cast("int").alias("Quantity"),
        col("oi.CurrentPrice").cast("double").alias("CurrentPrice"),
        col("inv.UnitCost").cast("double").alias("UnitCost"),  # New column
        (col("oi.Quantity").cast("int") * col("oi.CurrentPrice").cast("double")).alias("Revenue"),
        (
            (col("oi.Quantity").cast("int") * col("oi.CurrentPrice").cast("double")) -
            (col("oi.Quantity").cast("int") * col("inv.UnitCost").cast("double"))
        ).alias("Profit"),  # New column
        col("p.CreatedAt").alias("CreatedAt")  # Use payment time
    )
)
fact_sales.write.jdbc(mysql_url, "FactSales", mode="overwrite", properties=mysql_props)
# fact_sales.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/FactSales")

In [ ]:
# Fact reasons
reasons = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Reasons")
orders = spark.read.parquet("wasbs://silver@mysilver.blob.core.windows.net/Orders")

# Join Reasons with Orders to get OrderDateKey and StatusID
fact_order_reason = (
    reasons
    .join(orders, reasons.OrderID == orders.OrderID, "inner")
    .select(
        reasons.ReasonID,
        reasons.OrderID,
        date_format(orders.CreatedAt, "yyyyMMdd").cast("int").alias("OrderDateKey"),
        orders.StatusID,
        reasons.CreatedAt.alias("CreatedAt")  # Use Reasons.CreatedAt
    )
)
fact_order_reason.write.jdbc(mysql_url, "FactOrderReason", mode="append", properties=mysql_props)
# fact_order_reason.write.mode("overwrite").parquet("abfss://gold-test@mygold.dfs.core.windows.net/FactOrderReason")